<a href="https://colab.research.google.com/github/sheshankcool/PyDeequ/blob/main/PyDeequ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [4]:
import findspark
findspark.init()


In [5]:

!pip install pydeequ

In [6]:
!pip install pyspark==3.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 MB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 14.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767581 sha256=b869fb91e65eb38cee708c4ba33398e98e3a6fabbf5cc40b19d6769de9769278
  Stored in directory: /root/.cache/pip/wheels/a0/3f/72/8efd988f9ae041f051c75e6834cd92dd6d13a726e206e8b6f3
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.3
    Uninstalling pyspark-3.5.3:
      Successfully uninstalled pyspark-3.5.3


In [7]:
import os
os.environ['SPARK_VERSION'] = '3.1.1'

In [8]:
from pyspark.sql import SparkSession, Row
import pydeequ
spark = (SparkSession
             .builder
             .config("spark.jars.packages", pydeequ.deequ_maven_coord)
             .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
             .getOrCreate())

In [9]:

df = spark.sparkContext.parallelize([
    Row(a="foo", b=1, c=5, d=10, e=None, f=0),
    Row(a="bar", b=2, c=6, d=4, e= 12, f=90),
    Row(a="baz", b=3, c=None, d=None, e = 20, f= -10),
    Row(a="cab", b=3, c=8,  d=None, e =None, f=50)]).toDF()


In [10]:
from pydeequ.checks import *
from pydeequ.verification import *
check = Check(spark, CheckLevel.Warning, "Basic Check")

In [11]:

#

In [12]:

checkResult = VerificationSuite(spark) \
 .onData(df) \
 .addCheck(check.hasSize(lambda x: x == 4).hasSize(lambda x:x<=2))\
 .run()


Python Callback server started!


In [13]:
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

+-----------+-----------+------------+--------------------------+-----------------+--------------------------------------------------+
|check      |check_level|check_status|constraint                |constraint_status|constraint_message                                |
+-----------+-----------+------------+--------------------------+-----------------+--------------------------------------------------+
|Basic Check|Warning    |Warning     |SizeConstraint(Size(None))|Success          |                                                  |
|Basic Check|Warning    |Warning     |SizeConstraint(Size(None))|Failure          |Value: 4 does not meet the constraint requirement!|
+-----------+-----------+------------+--------------------------+-----------------+--------------------------------------------------+



#### check if the number of columns are as the required number

In [14]:
check = Check(spark, CheckLevel.Warning, "Basic Check")
checkResult = VerificationSuite(spark) \
 .onData(df) \
 .addCheck(check.hasSize(lambda x: x == 5))\
 .run()
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)


+-----------+-----------+------------+--------------------------+-----------------+--------------------------------------------------+
|check      |check_level|check_status|constraint                |constraint_status|constraint_message                                |
+-----------+-----------+------------+--------------------------+-----------------+--------------------------------------------------+
|Basic Check|Warning    |Warning     |SizeConstraint(Size(None))|Failure          |Value: 4 does not meet the constraint requirement!|
+-----------+-----------+------------+--------------------------+-----------------+--------------------------------------------------+



### check if there are missing values in given columns
- value is 0.75 because it divides non null rows/total rows since 1/4 which is 0.75 which means that only 75% are not null but not 100% which is 1.0. here it will compare the division output with 1 if it is not equal it will raise error.

In [15]:
check = Check(spark, CheckLevel.Warning, "Basic Check")
checkResult = VerificationSuite(spark) \
 .onData(df) \
 .addCheck(check.isComplete('a')\
           .isComplete('b')\
           .isComplete('c')\
           .isComplete('d')\
           .isComplete('e')\
           .isComplete('f'))\
 .run()
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

+-----------+-----------+------------+-------------------------------------------------+-----------------+-----------------------------------------------------+
|check      |check_level|check_status|constraint                                       |constraint_status|constraint_message                                   |
+-----------+-----------+------------+-------------------------------------------------+-----------------+-----------------------------------------------------+
|Basic Check|Warning    |Warning     |CompletenessConstraint(Completeness(a,None,None))|Success          |                                                     |
|Basic Check|Warning    |Warning     |CompletenessConstraint(Completeness(b,None,None))|Success          |                                                     |
|Basic Check|Warning    |Warning     |CompletenessConstraint(Completeness(c,None,None))|Failure          |Value: 0.75 does not meet the constraint requirement!|
|Basic Check|Warning    |Warning  

### checking for nullvalues on multiplecolumns at a time
- where there should not be null values in all the given columns, it raises error if there is null value in atlease one of given column

In [16]:
check = Check(spark, CheckLevel.Warning, "Basic Check")
checkResult = VerificationSuite(spark) \
 .onData(df) \
 .addCheck(check.areComplete(['a','b']))\
 .addCheck(check.areComplete(['a','c','d']))\
 .run()
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

+-----------+-----------+------------+-------------------------------------------------------------------------------------------------------------------------------------------+-----------------+----------------------------------------------------+
|check      |check_level|check_status|constraint                                                                                                                                 |constraint_status|constraint_message                                  |
+-----------+-----------+------------+-------------------------------------------------------------------------------------------------------------------------------------------+-----------------+----------------------------------------------------+
|Basic Check|Warning    |Success     |ComplianceConstraint(Compliance(Combined Completeness,((a IS NOT NULL) AND (b IS NOT NULL)),None,List(a, b),None))                         |Success          |                                                    |


In [17]:
df.show()

+---+---+----+----+----+---+
|  a|  b|   c|   d|   e|  f|
+---+---+----+----+----+---+
|foo|  1|   5|  10|null|  0|
|bar|  2|   6|   4|  12| 90|
|baz|  3|null|null|  20|-10|
|cab|  3|   8|null|null| 50|
+---+---+----+----+----+---+



### hasCompleteness
- will check what is the percentage of non null columns

In [18]:
from pydeequ.checks import *
from pydeequ.verification import *
check = Check(spark, CheckLevel.Warning, "Basic Check")
checkResult = VerificationSuite(spark) \
 .onData(df) \
 .addCheck(check.hasCompleteness('a',lambda x: x >= 0.8))\
 .run()
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

+-----------+-----------+------------+-------------------------------------------------+-----------------+------------------+
|check      |check_level|check_status|constraint                                       |constraint_status|constraint_message|
+-----------+-----------+------------+-------------------------------------------------+-----------------+------------------+
|Basic Check|Warning    |Success     |CompletenessConstraint(Completeness(a,None,None))|Success          |                  |
+-----------+-----------+------------+-------------------------------------------------+-----------------+------------------+



In [19]:
from pydeequ.checks import *
from pydeequ.verification import *
check = Check(spark, CheckLevel.Warning, "Basic Check")
checkResult = VerificationSuite(spark) \
 .onData(df) \
 .addCheck(check.hasCompleteness('c',lambda x: x >= 0.75))\
 .run()
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

+-----------+-----------+------------+-------------------------------------------------+-----------------+------------------+
|check      |check_level|check_status|constraint                                       |constraint_status|constraint_message|
+-----------+-----------+------------+-------------------------------------------------+-----------------+------------------+
|Basic Check|Warning    |Success     |CompletenessConstraint(Completeness(c,None,None))|Success          |                  |
+-----------+-----------+------------+-------------------------------------------------+-----------------+------------------+



In [20]:
from pydeequ.checks import *
from pydeequ.verification import *
check = Check(spark, CheckLevel.Warning, "Basic Check")
checkResult = VerificationSuite(spark) \
 .onData(df) \
 .addCheck(check.hasCompleteness('c',lambda x: x >= 0.75))\
 .run()
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

+-----------+-----------+------------+-------------------------------------------------+-----------------+------------------+
|check      |check_level|check_status|constraint                                       |constraint_status|constraint_message|
+-----------+-----------+------------+-------------------------------------------------+-----------------+------------------+
|Basic Check|Warning    |Success     |CompletenessConstraint(Completeness(c,None,None))|Success          |                  |
+-----------+-----------+------------+-------------------------------------------------+-----------------+------------------+





### check for duplicates


In [21]:
check = Check(spark, CheckLevel.Warning, "Basic Check")
checkResult = VerificationSuite(spark) \
 .onData(df) \
 .addCheck(check.isUnique('a'))\
 .addCheck(check.isUnique('b'))\
 .run()
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

+-----------+-----------+------------+---------------------------------------------------+-----------------+----------------------------------------------------+
|check      |check_level|check_status|constraint                                         |constraint_status|constraint_message                                  |
+-----------+-----------+------------+---------------------------------------------------+-----------------+----------------------------------------------------+
|Basic Check|Warning    |Success     |UniquenessConstraint(Uniqueness(List(a),None,None))|Success          |                                                    |
|Basic Check|Warning    |Warning     |UniquenessConstraint(Uniqueness(List(a),None,None))|Success          |                                                    |
|Basic Check|Warning    |Warning     |UniquenessConstraint(Uniqueness(List(b),None,None))|Failure          |Value: 0.5 does not meet the constraint requirement!|
+-----------+-----------+---

In [22]:
check = Check(spark, CheckLevel.Warning, "Basic Check")
checkResult = VerificationSuite(spark) \
 .onData(df) \
 .addCheck(check.hasUniqueness(['a'],lambda x : x > 0.75))\
 .run()
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)



+-----------+-----------+------------+--------------------------------------------------------+-----------------+------------------+
|check      |check_level|check_status|constraint                                              |constraint_status|constraint_message|
+-----------+-----------+------------+--------------------------------------------------------+-----------------+------------------+
|Basic Check|Warning    |Success     |UniquenessConstraint(Uniqueness(Stream(a, ?),None,None))|Success          |                  |
+-----------+-----------+------------+--------------------------------------------------------+-----------------+------------------+



In [23]:
df.show()

+---+---+----+----+----+---+
|  a|  b|   c|   d|   e|  f|
+---+---+----+----+----+---+
|foo|  1|   5|  10|null|  0|
|bar|  2|   6|   4|  12| 90|
|baz|  3|null|null|  20|-10|
|cab|  3|   8|null|null| 50|
+---+---+----+----+----+---+



In [24]:
check = Check(spark, CheckLevel.Warning, "Basic Check")
checkResult = VerificationSuite(spark) \
 .onData(df) \
 .addCheck(check.hasMin('b',lambda x : x <= 2))\
 .run()
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

+-----------+-----------+------------+---------------------------------------+-----------------+------------------+
|check      |check_level|check_status|constraint                             |constraint_status|constraint_message|
+-----------+-----------+------------+---------------------------------------+-----------------+------------------+
|Basic Check|Warning    |Success     |MinimumConstraint(Minimum(b,None,None))|Success          |                  |
+-----------+-----------+------------+---------------------------------------+-----------------+------------------+



In [25]:
check = Check(spark, CheckLevel.Warning, "Basic Check")
checkResult = VerificationSuite(spark) \
 .onData(df) \
 .addCheck(check.hasMin('f',lambda x : x >= 1))\
 .run()
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

+-----------+-----------+------------+---------------------------------------+-----------------+------------------------------------------------------+
|check      |check_level|check_status|constraint                             |constraint_status|constraint_message                                    |
+-----------+-----------+------------+---------------------------------------+-----------------+------------------------------------------------------+
|Basic Check|Warning    |Warning     |MinimumConstraint(Minimum(f,None,None))|Failure          |Value: -10.0 does not meet the constraint requirement!|
+-----------+-----------+------------+---------------------------------------+-----------------+------------------------------------------------------+



In [26]:
check = Check(spark, CheckLevel.Warning, "Basic Check")
checkResult = VerificationSuite(spark) \
 .onData(df) \
 .addCheck(check.hasMax('f',lambda x : x <= 100))\
 .run()
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

+-----------+-----------+------------+---------------------------------------+-----------------+------------------+
|check      |check_level|check_status|constraint                             |constraint_status|constraint_message|
+-----------+-----------+------------+---------------------------------------+-----------------+------------------+
|Basic Check|Warning    |Success     |MaximumConstraint(Maximum(f,None,None))|Success          |                  |
+-----------+-----------+------------+---------------------------------------+-----------------+------------------+



In [28]:
check = Check(spark, CheckLevel.Warning, "Basic Check")
checkResult = VerificationSuite(spark) \
 .onData(df) \
 .addCheck(check.isPositive('a').isNonNegative('a').isPositive('b').isNonNegative('b')) \
 .run()
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)


+-----------+-----------+------------+-------------------------------------------------------------------------------------------------------------------+-----------------+------------------+
|check      |check_level|check_status|constraint                                                                                                         |constraint_status|constraint_message|
+-----------+-----------+------------+-------------------------------------------------------------------------------------------------------------------+-----------------+------------------+
|Basic Check|Warning    |Success     |ComplianceConstraint(Compliance(a is positive,COALESCE(CAST(a AS DECIMAL(20,10)), 1.0) > 0,None,List(a),None))     |Success          |                  |
|Basic Check|Warning    |Success     |ComplianceConstraint(Compliance(a is non-negative,COALESCE(CAST(a AS DECIMAL(20,10)), 0.0) >= 0,None,List(a),None))|Success          |                  |
|Basic Check|Warning    |Success     |Co

In [29]:
check = Check(spark, CheckLevel.Warning, "Basic Check")
checkResult = VerificationSuite(spark) \
 .onData(df) \
 .addCheck(check.isContainedIn("a", ["foo", "bar", "baz"]))\
 .run()
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

+-----------+-----------+------------+------------------------------------------------------------------------------------------------------------------------+-----------------+-----------------------------------------------------+
|check      |check_level|check_status|constraint                                                                                                              |constraint_status|constraint_message                                   |
+-----------+-----------+------------+------------------------------------------------------------------------------------------------------------------------+-----------------+-----------------------------------------------------+
|Basic Check|Warning    |Warning     |ComplianceConstraint(Compliance(a contained in foo,bar,baz,`a` IS NULL OR `a` IN ('foo','bar','baz'),None,List(a),None))|Failure          |Value: 0.75 does not meet the constraint requirement!|
+-----------+-----------+------------+----------------------------------

In [31]:
check = Check(spark, CheckLevel.Warning, "Basic Check")
checkResult = VerificationSuite(spark) \
 .onData(df) \
 .addCheck(check.hasDataType("c",ConstrainableDataTypes.Numeric))\
 .run()
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

+-----------+-----------+------------+----------------------------------------------------------------------------+-----------------+------------------+
|check      |check_level|check_status|constraint                                                                  |constraint_status|constraint_message|
+-----------+-----------+------------+----------------------------------------------------------------------------+-----------------+------------------+
|Basic Check|Warning    |Success     |AnalysisBasedConstraint(DataType(c,None),<function1>,Some(<function1>),None)|Success          |                  |
+-----------+-----------+------------+----------------------------------------------------------------------------+-----------------+------------------+

